In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [5]:
#constants
pi = np.pi;
e = np.e;

### FFT

In [108]:
def fft(P):

    if(len(P) == 1):
        return P;
    

    P_e = P[0:: 2];
    P_o = P[1:: 2];

    y_e = fft(P_e);
    y_o = fft(P_o);

    y = np.zeros(len(P), dtype=np.complex_);
    w = e**(1j * 2*pi/len(P));
    w0 = 1;

    for i in range(len(y_e)):
        y[i] = w0 * y_o[i] + y_e[i];
        y[i+len(y_e)] = -w0 * y_o[i] + y_e[i];
        w0 *= w;
    

    return y;

def ifft(P):

    if(len(P) == 1):
        return P;
    

    P_e = P[0:: 2];
    P_o = P[1:: 2];

    y_e = fft(P_e);
    y_o = fft(P_o);

    y = np.zeros(len(P), dtype=np.complex_);
    w = e**(-1j * 2*pi/len(P));
    w0 = 1;

    for i in range(len(y_e)):
        y[i] = w0 * y_o[i] + y_e[i];
        y[i+len(y_e)] = -w0 * y_o[i] + y_e[i];
        w0 *= w;
    

    return y/len(P);

def fpad(P):

    n = int(2**(np.ceil(np.log2(len(P)))));
    if(len(P)-n==0):
        return P;

    padded = [];
    for i in range(0,  len(P)):
        padded.append(P[i]);
    
    for i in range(len(P), n):
        padded.append(0);

    return np.array(padded, dtype = np.complex_);

In [127]:
P = [-1 + 0j, 0 + 0j, 1 + 0j]
ifft(fft(fpad(P)))

array([-1.+0.j,  0.+0.j,  1.+0.j,  0.+0.j])

### Newton Raphson

In [125]:
def diff(P):

    dP = [];
    for i in range(1, len(P)):
        dP.append(i*P[i]);
    dP.append(0);
    dP = np.array(dP, dtype=np.complex_);

    return dP;

def remove_trailing_zeros(P):

    n = 0;
    for i in range(len(P)-1, 0, -1):
        if(np.abs(P[i]) > 0.000000001):
            n = i;
            break;

    return P[:n+1];

def poly_div(P, Q): #assuming Q is linear and should (somewhat) divide P
    Q = np.concatenate((Q, [0]*(len(P)-len(Q))));

    vP = fft(fpad(P));
    vQ = fft(fpad(Q));


    #locate zero of Q
    n = -1;
    for i in range(len(vQ)):
        if(np.abs(vQ[i]) < 0.0000000000000001):
            n = i;
            break;

    #divide values

    h = [];

    m = len(vP);
    woff = e**(1j * 2*pi*n / m) + 0.00001 + 1j * 0.00001;

    for i in range(m):

        if(i==n):
            pows = [woff];
            wi = woff;
            for k in range(1, len(P)):
                wi *= woff;
                pows.append(wi);
            
            h.append(P.dot(pows)/(Q.dot(pows)));
        
        else:
            h.append(vP[i]/vQ[i]);

    return remove_trailing_zeros(ifft(h));


def all_roots(P, niter = 20):

    #create derivative
    P = np.array(P, dtype = np.complex_);
    dP = diff(P);

    #initial guess
    n = len(P);
    roots = [];

    for i in range(n-1):
        x0 = np.random.uniform(-1, 1);    
        y0 = np.random.uniform(-1, 1);
        z0 = x0 + 1j * y0;

        for j in range(niter):
            pows = [z0];
            zi = z0;
            for k in range(1, len(P)):
                zi *= z0;
                pows.append(zi);

            z0 -= P.dot(pows)/dP.dot(pows);

        P = poly_div(P, [-z0, 1]);
        dP = diff(P);
        roots.append(z0);

    return roots;

In [105]:
#specific newton raphson implementation

def calc(z0, n):
    fz = [z0]
    for i in range(n-1):
        z0 = z0*z0+z0;
        fz.append(z0);
    return np.array(fz);

def spec_newt_raph(z0, n, niter = 20):

    for i in range(niter):
        fz = calc(z0, n);
        prod = 1;
        for z in fz:
            prod *= (2*z+1);
        z0 -= (fz[-1]**2+fz[-1])/prod;
    
    return z0

In [145]:
spec_newt_raph(-1.39, 4)

-1.0

### Main Event

In [ ]:
def genady_polynomial(c1, p, q):

    genady = [1];

    #start by calculating the inner summands, and keep them in an array
    derv = 1/(2*c1);
    z0 = c1;


    derivs = [derv];

    for i in range(2, p*q+1):
        z0 = z0*z0 + c1;
        if(np.abs(derv)<0.00000000000000001):
            derv = 0;
        
        else:
            derv = derv/(2*z0);
        
        derivs.append(derv);
    
    #now calculate polynomial
    for n in range(1, q):
        genady.append(sum(derivs[p*(n-1)+1 : p*n+1]));
    

    genady.append(sum(derivs[p*(q-1)+1 : p*q]));

    return genady;

In [102]:
np.concatenate((np.array([1,2,3]), [0]*10)).dot([0]*13)

0